### Predicting Mask on Augmented Dataset

1. Load Model
2. Load Test Dataset - Images
3. Predict Mask
4. Calculate FPS

In [2]:
import os
import time
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf

#### Seeding

In [5]:
# Seeding
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

#### Hyperparameters

In [4]:
# Hyperparameters
height = 768
width = 512
num_classes = 3

#### Path

In [ ]:
# Paths
dataset_path = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/processed_dataset/aug/test"
save_path = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/predicted_masks/aug"
model_path = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/files/aug/unet-multiclass.h5"

#### Create Folder

In [ ]:
os.makedirs(save_path, exist_ok=True)

#### Folder to save the predicted masks

In [ ]:
# create_dir(save_path)

#### Load Model

In [ ]:
model = tf.keras.models.load_model(model_path)

In [ ]:
model.summary()

#### Load Test Dataset: Images

In [ ]:
test_x = sorted(glob(os.path.join(dataset_path, "images", "*")))

# print(f"Total test images: {len(test_x)}")

In [ ]:
# test_x[0]

#### Predict the mask and calculate FPS

In [ ]:
time_taken = []

for x_path in tqdm(test_x, desc="Predicting Masks"):
    name = os.path.basename(x_path)
    x = cv2.imread(x_path, cv2.IMREAD_COLOR)
    x = x / 255.0
    x = np.expand_dims(x, axis=0)

    start_time = time.time()
    p = model.predict(x)[0]
    time_taken.append(time.time() - start_time)

    p = np.argmax(p, axis=-1)  # Convert probabilities to class labels
    p_colored = cv2.applyColorMap((p * 85).astype(np.uint8), cv2.COLORMAP_JET)  # Visualize classes

    cv2.imwrite(os.path.join(save_path, name), p_colored)

In [ ]:
# FPS Calculation
mean_time = np.mean(time_taken)
mean_fps = 1 / mean_time

In [ ]:
print(f"Mean time taken: {mean_time} seconds")
print(f"Mean FPS: {mean_fps}")